<a href="https://colab.research.google.com/github/valentingracia/codingDojoDC/blob/main/Tarea_Arboles_Semana6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tarea Semana 6

In [95]:
#Librerias clasicas

#Models
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
#Bagged Trees and Random Forest
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor

from sklearn.datasets import load_boston
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd

#Widgets
from ipywidgets import interact, interactive, fixed, interact_manual,IntSlider
import ipywidgets as widgets

#explore df
import missingno as msno 
#other
from termcolor import colored
from google.colab import output


#Chocopanda

In [96]:
kcHouseData_df=pd.read_csv("/content/drive/MyDrive/Coding Dojo/Semana6/kc_house_data.csv")
kcHouseData_df.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639


EXploracion

In [97]:
kcHouseData_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

#body Counts

In [98]:
from google.colab import widgets

#estoy trabajando esta funcion.
def f(varCol=0):
  df=kcHouseData_df
  output.clear()
  f=df.iloc[:,[varCol]];
  f=f.value_counts()
  f=pd.DataFrame(f)
  #print(f"csm {df.iloc[:,[x]].columns}")
  #print("Variable ", colored(f"*{df.iloc[:,[x]].column}*",'red'))
  #f.index.name =df.iloc[:,[x]].columns
  f.columns = ['count']

  print("Variable ", colored(f"*{varCol}*  dtype:{df.dtypes[varCol]}",'red'))
  print("-----")
  print(f"{len(f.index)} Categorias")
  print(f"{df.shape[0]-f.values.sum()} perdidos de  {df.shape[0]}")
  if len(f.index)== df.shape[0]:
    print("UNICO, se podria usar de indice")
 
 
  return f;
  
interact(f, varCol=(0,len(kcHouseData_df.columns)-1))

interactive(children=(IntSlider(value=0, description='varCol', max=20), Output()), _dom_classes=('widget-inter…

<function __main__.f>

#Preparamos dataset para facilitar su proceso

In [99]:
kcHouseData_df.index=kcHouseData_df["id"]
kcHouseData_df.drop(columns=["date","id"],inplace=True)

In [100]:
X=kcHouseData_df.drop(columns=["price"])
y=kcHouseData_df["price"]

In [69]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=3)

In [101]:
bt=BaggingRegressor()
bt.fit(X_train,y_train)

BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=1.0, max_samples=1.0, n_estimators=10,
                 n_jobs=None, oob_score=False, random_state=None, verbose=0,
                 warm_start=False)

In [102]:
#training R2
bt.score(X_train,y_train)

0.975225015768655

In [103]:
#Testing R2
bt.score(X_test,y_test)

0.8698847803046511

Ahora Random Forest

In [104]:
rf=RandomForestRegressor()
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [105]:
#training R2
rf.score(X_train,y_train)

0.9816849671409719

In [106]:
#test R2
rf.score(X_test,y_test)

0.884892359562453

Cuales la columna mas relevante

In [76]:
rf_features=pd.DataFrame({'feature':X_train.columns,"feature_importance":rf.feature_importances_})
rf_features.sort_values(by="feature_importance",inplace=True)

In [107]:
rf_features

,feature,feature_importance
12,yr_renovated,0.002181
4,floors,0.002197
7,condition,0.002704
0,bedrooms,0.003190
10,sqft_basement,0.005781
1,bathrooms,0.006060
13,zipcode,0.012709
17,sqft_lot15,0.013258
3,sqft_lot,0.013813
6,view,0.017185


Latitud,metros cuadrados y grado.

In [ ]:
plt.figure(figsize=(12,5))
plt.barh(rf_feature["feature"],rf_feature["feature_importance"]);

#que otros parametros pudimos tunear

In [108]:
#param_grid = [{'max_depth': [2,3,4,5,6,7,8,9,10], 'n_estimators': [10,20,30,40,50,60,70,80,90,100], 'max_samples': [0.05,0.1,0.15]}]
param_grid = [{'n_estimators': [10,20,30,40,50,60,70,80,90,100]}]


In [109]:
rf= GridSearchCV(estimator =RandomForestRegressor() , param_grid = param_grid, scoring = 'neg_mean_squared_error', cv=10) 


In [ ]:
rf.fit(X_train, y_train)

In [94]:
print("Mejor score (error minimo cuadrado negativo): ", rf.best_score_)
print("Mejores hiperparámetros: ", rf.best_params_)
print("Mejor modelo: ", rf.best_estimator_)

model = rf.best_estimator_

Mejor score (error minimo cuadrado negativo):  -46957913851.86029
Mejores hiperparámetros:  {'max_depth': 3}
Mejor modelo:  RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=3, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)


In [90]:
rf.score(X_test,y_test)

-30197379419.928486

No logre interpretar el error negativo cuadrado, el cual se disaro.